# Workspace for Data Compression Tasks
[Click here for the source repository](https://github.com/MarkGotham/Data_Compression)

In [2]:
from IPython.display import Markdown
Markdown("./tasks/fourier_roundtrip.md")  # Load the task text

## Background

For data compression to be effective,
we need to 'encode' the data to a small representation size 
and then 'decode' it back to the original (or nearly).

A classic way to test the effectiveness of a system like this is to run a 
'roundtrip' where you compare the data you star with to what comes back from an encoding-decoding cycle. 


## Task

- Type: Implement roundtrip
- Task:
  - Import or create some synthetic data, like an 8x8 array with a continuous tone pattern.
  - Import an algorithm for the fourier transform (e.g., `import scipy.fft as fft`) 
    - (Bonus: code it from scratch)
  - Take the source data, run the fft on that source, and then the inverse fft on the fft output.
  - Compare the final data with the original.
- Reference implementation in the notebook: `test_roundtrip()`


## Also on this repository

The [`jpeg` notebook](https://github.com/MarkGotham/Data_Compression/blob/main/jpeg.ipynb)
runs a similar roundtrip, with the following differences:
- the additional consideration of quantization,
- the use of DCT,
- additional abstraction (local implementations on the repo).


## Workspace

## Reference

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.fft as fft

First, let's create an 8-by-8 matrix,
and fill it with a continuous tone pattern.

Don't worry about the code,
the image will show the resulting pattern.

In [ ]:
continuous_tone_pattern = np.zeros((8,8))        

In [ ]:
continuous_tone_pattern

In [ ]:
for m in range(continuous_tone_pattern.shape[0]):
    for n in range(continuous_tone_pattern.shape[1]):
        continuous_tone_pattern[m][n] = abs(3.5 - m) * abs(3.5 - n) * 20

In [ ]:
continuous_tone_pattern

In [ ]:
plt.imshow(continuous_tone_pattern)
plt.show()

Now perform the DCT function ...

In [ ]:
cont_dct = fft.dctn(continuous_tone_pattern, norm="ortho")
cont_dct.round(3)

... look at all those 0s! Lots of (opportunity for) compression here.

... assuming we can get back to the original ...

... So now we run the inverse to see how close it is

In [ ]:
inverse = fft.idctn(cont_dct, norm="ortho").round(2)
inverse

Does this array look familiar? Let's check ... 

In [ ]:
def test_roundtrip(
        a: np.array
) -> np.array:
    """
    Implements a roundtrip test on a 2-d array and 
    returns the value and position of the largest divergence.
    I.e.,
    - Take an array (a),
    - run DCT,
    - quant (to nearest integer in this case),
    - run IDCT,
    - compare change on each item (abs diff in this case).
    """
    dct_round = fft.dctn(a, norm="ortho").round(0)
    idct_v = fft.idctn(dct_round, norm="ortho")
    m, n = a.shape
    assert m > 1
    assert n > 1
    max_diff = 0
    diff_ref = None
    for i in range(m):
        for j in range(n):
            this_diff = abs(a[i][j] - idct_v[i][j])
            if this_diff > max_diff:
                max_diff = this_diff
                diff_ref = (i, j)

    return round(max_diff, 2), diff_ref

In [ ]:
test_roundtrip(continuous_tone_pattern)

This (above ^^^) is the maximum difference of any pixel in the 8-by-8 array along with its position in the array.

And here is that value as a percentage ...

In [ ]:
val, idx = test_roundtrip(inverse)

round(100 * val / continuous_tone_pattern[idx[0], idx[1]], 2)